### General Usage of the Notebook: 

The following code will perform the following action:

(1) Read the downloaded images of SVHN

(2) Using the blue bounding box marked on the digits of street number within each individual image, find the bounding box exclosing the full street number sequence

(3) Extend the bounding box by 30% and crop the image to the size of the bounding box

(4) Resize the image to size of (54,54,3).

### Data Source & Description:

SVHN is a real-world image dataset for developing machine learning and object recognition algorithms with minimal requirement on data preprocessing and formatting. The dataset is similar in flavor to MNIST but incorporates an order of magnitude more labeled data (over 600,000 digit images) and comes from a significantly harder, unsolved, real world problem. SVHN is obtained from house numbers in Google Street View images. There are three datasets available, train data (size 33401), test data (size 13068), and extra dataset (to be used if more data is needed for training). The datasets are provided in zip file.


source: http://ufldl.stanford.edu/housenumbers/

### Final Output:

The final output of this jupyter notebook gives one pickle file named "trainpkl.gz". This pickle file will be loaded during actual training to save the time from reading image directly.


In [1]:
# Total Number of image is 33401, the file name start from 1 to 33402
Train_no = 33402

In [2]:
#Before running, all the images need to be extracted from the zip folder downloaded from the website.
#This code only reads the training image from un-zipped folder.

from scipy import misc
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

X_train=[]
#Make sure you change directory to your local directory where the unzipped images are stored
directory = "./train/"
img_type=".png"
for i in range(1,Train_no):
    image_name=[i,img_type]
    values = ''.join(str(v) for v in image_name)
    folder=[directory,values] 
    folder=''.join(folder)
    image = misc.imread(folder)
    X_train.append(image)


/Users/Shimeng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


In [3]:
#This is the function to get the blue box position from the mat file provided with the training image
def get_box_data(index, hdf5_data):
    
    meta_data = dict()
    meta_data['height'] = []
    meta_data['label'] = []
    meta_data['left'] = []
    meta_data['top'] = []
    meta_data['width'] = []

    def print_para(name, obj):
        vals = []
        if obj.shape[0] == 1:
            vals.append(obj[0][0])
        else:
            for k in range(obj.shape[0]):
                vals.append(int(hdf5_data[obj[k][0]][0][0]))
        meta_data[name] = vals

    box = hdf5_data['/digitStruct/bbox'][index]
    hdf5_data[box[0]].visititems(print_para)
    return meta_data

def get_para(index, hdf5_data):
    name = hdf5_data['/digitStruct/name']
    return ''.join([chr(v[0]) for v in hdf5_data[name[index][0]].value])

In [4]:
# Run this cell will be able to provide you the cropped image in 54*54*3 format for training dataset
import h5py
import matplotlib.pyplot as plt
import cv2

#Make sure you change directory to your local directory where the mat files are stored
#.mat file location
mat_data = h5py.File('./train/digitStruct.mat')
size = mat_data['/digitStruct/name'].size

label_Final = []
Pic_crop_Final = []
#obtain label and basic parameters of each image
for i in range(Train_no-1):
    Array = np.array(X_train[i])
    pic = get_para(i, mat_data)
    box = get_box_data(i, mat_data)
    label = box['label']
    label_Final.append(''.join(str(int(x%10)) for x in label))

    #Increase location by 30 on any edge of the box%
    
    H = int(round((max(box['top'])+max(box['height'])) *1.03))
    L = int(round (min(box['left']) *(0.97)))
    #ensure there is no negative dimension due to some of the picture have the weird locations
    if L<0:
        L =0
    T = int(round (min(box['top'])*(0.97)))
    if T<0:
        T = 0
    W = int(round((max (box ['left'])+ max(box['width']))*1.03))
    #Crop the image
    Pic_crop = Array [T:H,L:W]

    #resize data and append to the created list

    res_Pic_crop = cv2.resize(Pic_crop, dsize=(54, 54), interpolation=cv2.INTER_CUBIC)
    Pic_crop_Final.append(res_Pic_crop)

    

In [6]:
#Reshape the data from 54,54,3 into 54*54*3, and convert both label and image into arrays

c = []
for i in range(Train_no-1):
    a = np.reshape(Pic_crop_Final[i],54*54*3)
    c.append(a)
c = np.asarray(c)
d = np.asarray(label_Final)

In [ ]:
#Create pickle file and close the file
import six.moves.cPickle as pickle
import os
import sys
import gzip

#Please change this directory to your local directory
data_dir = '/Users/Shimeng/Documents/Master_Columbia_2018/E4040 NN and Deep Learning/Final Project/'
output_file = 'trainpkl.gz'
out_path = os.path.join(data_dir, output_file)

#create document format
out = {}
out['labels'] = d
out['images'] = c

#save data
p = gzip.open(out_path, 'wb')
pickle.dump(out, p)
p.close()